In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os

from sqlalchemy import create_engine, Integer, Float, String, Date
from sqlalchemy.types import TypeEngine

In [3]:
sscard_df = pd.read_csv("../data/DB/lotte/002_ltmb_k7_data.csv")

In [4]:
sscard_df.shape

(4998619, 11)

In [7]:
17236888 / 4998619

3.448330028753942

In [5]:
4998619 * 11

54984809

In [8]:
sscard_df["stdt"] = pd.to_datetime(sscard_df["stdt"], format="%Y%m")

In [13]:
def get_column_type(col_name: str, dtype: str) -> TypeEngine:
    """컬럼 이름과 데이터 타입에 따라 적절한 SQLAlchemy 타입 반환"""
    if "int" in str(dtype):
        if col_name in ["stdt", "CRI_YM"]:  # 날짜 컬럼
            return Date
        elif df[col_name].max() < 2147483647 and df[col_name].min() > -2147483648:
            return Integer
        else:
            return Float  # bigint 대신 Float 사용
    elif "float" in str(dtype):
        return Float
    else:
        return String(length=255)  # 문자열 길이 제한

In [10]:
sscard_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998619 entries, 0 to 4998618
Data columns (total 11 columns):
 #   Column     Dtype         
---  ------     -----         
 0   stdt       datetime64[ns]
 1   channel    object        
 2   ch_region  object        
 3   bcode1     int64         
 4   bname1     object        
 5   count      int64         
 6   money      int64         
 7   ppl        int64         
 8   gender     object        
 9   age        int64         
 10  region     object        
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 419.5+ MB


In [14]:
for col in sscard_df.select_dtypes(include=["int64", "float64"]).columns:
    sscard_df[col] = pd.to_numeric(sscard_df[col], downcast="integer")

In [15]:
sscard_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998619 entries, 0 to 4998618
Data columns (total 11 columns):
 #   Column     Dtype         
---  ------     -----         
 0   stdt       datetime64[ns]
 1   channel    object        
 2   ch_region  object        
 3   bcode1     int8          
 4   bname1     object        
 5   count      int32         
 6   money      int32         
 7   ppl        int16         
 8   gender     object        
 9   age        int8          
 10  region     object        
dtypes: datetime64[ns](1), int16(1), int32(2), int8(2), object(5)
memory usage: 286.0+ MB


In [16]:
sscard_df["bcode1"]

0           1
1           1
2           3
3           3
4           3
           ..
4998614    37
4998615    50
4998616    51
4998617    52
4998618    59
Name: bcode1, Length: 4998619, dtype: int8

In [4]:
# concat data ltmb [0, 3]
lotte_df = pd.concat(
    [pd.read_csv(data_dict["ltmb"][0]), pd.read_csv(data_dict["ltmb"][3])]
)
lotte_df.head()
"""
dataframe description
stdt : 구매 년월 <거래 발생 년월>
channel : 구매 채널 <마트, 편의점 2개 채널 존재>
ch_region : 점포 위치 <소비자의 거래 발생 지역>
bcode1 : 구매 상품 대분류코드 <>
bname1 : 구매 상품 대분류명 <소비자의 구매 상품 대분류> (bcode1와 bname1 1:1 매칭)
count : 구매 건수 <몇개 구매했는지 의미하는 것으로 추정>
money : 구매 금액 <몇원 구매했는지 의미하는 것으로 추정>
ppl : 구매자 수 <몇명 구매했는지 의미하는 것으로 추정>
age : 구매자 연령대 <1대 이하 : 10, 20대 이하 : 20 ... 60 : 60대 이상>
region : 거주지 지역 <구매자의 거주지 지역>
"""

,stdt,channel,ch_region,bcode1,bname1,count,money,ppl,gender,age,region
0,202201,마트,강원도원주시,1,양곡,1,6990,1,m,40,충청북도충주시
1,202201,마트,강원도원주시,3,과일,6,66850,3,m,40,경기도안양시
2,202201,마트,강원도원주시,6,축산물,16,263108,7,w,30,경기도남양주시
3,202201,마트,강원도원주시,6,축산물,2,16390,2,w,30,울산광역시기타
4,202201,마트,강원도원주시,7,대용식,5,27430,3,m,20,서울특별시북서부


In [5]:
sscard_df.head()
"""
dataframe description
Dataframe의 줄은 1건의 거래가 발생할 때 마다 추가되는 것이다.
CRI_YM : 기준연월 <거래 발생 년월>
MRC_ADR : 가맹점소재지 <거래 발생 지역>
GENDER : 성별
AGE_GR : 연령대 <각 코드가 어디 연령대를 의미하는지 문의 필요함>
JOB_GR : 직업군 <소비자의 직업군>
CATEGORY_M_NM : 중분류 업종명 <가맹점 거래 발생 업종>
CATEGORY_L_NM : 대분류 업종명 <가맹점 거래 발생 업종>
MAIN_CSM_AREA : 소비자 주요 카드 사용 지역 <소비자 여행인지 아닌지 판단 가능할 것으로 보임>
"""

,CRI_YM,MRC_ADR,GENDER,AGE_GR,JOB_GR,CATEGORY_L_NM,CATEGORY_M_NM,MAIN_CSM_AREA,SUM_CNT,SUM_AMT
0,202201,경상남도창원시,1,C,02.회사원,생활,가례,경상북도남부,3.0,6106000.0
1,202201,충청남도남부,2,E,05.기타,생활,가례,충청남도남부,3.0,115000.0
2,202201,울산광역시남구,2,E,04.자영업자,생활,가례,울산광역시남구,3.0,99000.0
3,202201,대구광역시기타,2,C,05.기타,생활,가례,인천광역시기타,3.0,43000.0
4,202201,울산광역시남구,2,B,04.자영업자,생활,가례,인천광역시기타,3.0,1365000.0


In [10]:
sscard_df["CATEGORY_M_NM"].unique()

array(['가례', '가전', '관람', '교통', '병원', '보험', '뷰티', '서적', '숙박', '유흥', '의류',
       '주유', '주차', '취미', '학교', '학원', '골프장', '공과금', '면세점', '백화점', '아울렛',
       '음식점', '제과점', '편의점', '화장품', '건강보조', '교육기타', '놀이공원', '대중교통', '동물병원',
       '여행상품', '유아교육', '인테리어', '정기결제', '차량관리', '차량구매', '패션잡화', '커피/음료',
       '스포츠시설', '온라인쇼핑', '주류판매점', '주생활관리', '마트/할인점', '문구사무용품', '스포츠의류용품',
       '쇼핑기타', '상품권'], dtype=object)

In [11]:
sscard_df["CATEGORY_L_NM"].unique()

array(['생활', '쇼핑', '여가', '여행', '건강', '교육', '유흥', '자동차', '요식'],
      dtype=object)

In [12]:
sscard_df["JOB_GR"].unique()

array(['02.회사원', '05.기타', '04.자영업자', '03.공무원_교육인', '01.전문직'], dtype=object)

In [13]:
sscard_df["MRC_ADR"].unique()

array(['경상남도창원시', '충청남도남부', '울산광역시남구', '대구광역시기타', '충청남도천안시', '광주광역시기타',
       '경상북도포항시', '인천광역시기타', '충청북도청주시', '인천광역시남동구', '충청남도아산시', '경상북도남부',
       '전라남도여수시', '경상남도진주시', '부산광역시서부', '대구광역시수성구', '대구광역시동구', '대전광역시서구',
       '부산광역시사하구', '대구광역시달서구', '대전광역시기타', '세종특별자치시', '인천광역시서구', '광주광역시북구',
       '경상남도서부', '경상남도동부', '경상북도북부', '부산광역시동부', '대구광역시북구', '광주광역시광산구',
       '충청남도북부', '충청북도북부', '경상북도구미시', '경상남도김해시', '인천광역시부평구', '충청북도충주시',
       '전라남도동부', '전라남도순천시', '대전광역시유성구', '전라남도서부', '경상남도양산시', '울산광역시기타',
       '충청북도남부', '부산광역시부산진구', '부산광역시해운대구', '제주특별자치도제주시', '제주특별자치도서귀포시',
       '강원도영서', '강원도원주시', '강원도춘천시', '강원도영동'], dtype=object)

In [15]:
sscard_df["MAIN_CSM_AREA"].unique()

array(['경상북도남부', '충청남도남부', '울산광역시남구', '인천광역시기타', '충청남도천안시', '서울특별시북서부',
       '경상북도포항시', '대구광역시달서구', '대구광역시수성구', '충청북도청주시', '경기도남부', '경기도성남시',
       '경기도평택시', '전라남도동부', '전라남도서부', '경상남도진주시', '경상남도창원시', '광주광역시기타',
       '광주광역시북구', '대구광역시동구', '대구광역시북구', '대전광역시기타', '부산광역시동부', '서울특별시남부',
       '전라남도여수시', '광주광역시광산구', '대전광역시유성구', '서울특별시강남구', '서울특별시양천구',
       '제주특별자치도제주시', '제주특별자치도서귀포시', '경기도고양시', '경상북도북부', '경상북도구미시',
       '경기도화성시', '경상남도김해시', '부산광역시서부', '충청남도아산시', '서울특별시송파구', '부산광역시부산진구',
       '부산광역시해운대구', '세종특별자치시', '울산광역시기타', '인천광역시부평구', '충청남도북부', '대전광역시서구',
       '경상남도서부', '대구광역시기타', '충청북도충주시', '충청북도북부', '전라남도순천시', '서울특별시강동구',
       '경기도안양시', '인천광역시서구', '서울특별시강서구', '경기도용인시', '서울특별시북동부', '경상남도양산시',
       '서울특별시노원구', '경기도수원시', '경상남도동부', '부산광역시사하구', '경기도북부', '경기도안산시',
       '서울특별시관악구', '경기도남양주시', '인천광역시남동구', '충청북도남부', '서울특별시은평구', '강원도영서',
       '강원도원주시', '강원도춘천시', '강원도영동'], dtype=object)

In [22]:
# convert to datetime
sscard_df["CRI_YM"] = pd.to_datetime(sscard_df["CRI_YM"], format="%Y%m")

In [25]:
print(
    "Time range of CRI_YM: ",
    sscard_df["CRI_YM"].min(),
    " ~ ",
    sscard_df["CRI_YM"].max(),
)

Time range of CRI_YM:  2022-01-01 00:00:00  ~  2023-12-01 00:00:00
